In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib  # Dùng để lưu và tải mô hình
import json
import os

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Tắt các cảnh báo không cần thiết
import warnings
warnings.filterwarnings('ignore')

In [2]:
PATH_PROCESSED = '../data/processed/'
PATH_MODELS = '../models/'
PATH_REPORTS_FIGURES = '../reports/figures/'

# Tạo thư mục nếu chưa tồn tại
os.makedirs(PATH_MODELS, exist_ok=True)
os.makedirs(PATH_REPORTS_FIGURES, exist_ok=True)

In [4]:
train_data = np.load(f'{PATH_PROCESSED}train_data.npz')
# npz may use different key names; handle both common variants
if 'features' in train_data and 'labels' in train_data:
    X_train = train_data['features']
    y_train = train_data['labels']
elif 'X' in train_data and 'y' in train_data:
    X_train = train_data['X']
    y_train = train_data['y']
else:
    raise KeyError(f"Unexpected keys in train_data: {list(train_data.keys())}")

# Tải dữ liệu validation
val_data = np.load(f'{PATH_PROCESSED}val_data.npz')
if 'features' in val_data and 'labels' in val_data:
    X_val = val_data['features']
    y_val = val_data['labels']
elif 'X' in val_data and 'y' in val_data:
    X_val = val_data['X']
    y_val = val_data['y']
else:
    raise KeyError(f"Unexpected keys in val_data: {list(val_data.keys())}")

# Tải class mapping
with open(f'{PATH_PROCESSED}class_mapping.json', 'r') as f:
    class_mapping = json.load(f)

In [5]:
# Đảo ngược mapping để lấy tên class từ index
class_names = {v: k for k, v in class_mapping.items()}
target_names = [class_names[i] for i in sorted(class_names)]

print(f"Kích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")
print(f"Kích thước X_val: {X_val.shape}")
print(f"Kích thước y_val: {y_val.shape}")
print(f"Các lớp (classes): {target_names}")

Kích thước X_train: (8226, 224, 224, 3)
Kích thước y_train: (8226,)
Kích thước X_val: (2742, 224, 224, 3)
Kích thước y_val: (2742,)
Các lớp (classes): ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22']


In [6]:
# Khởi tạo các mô hình
models = {
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000),
    "Random Forest": RandomForestClassifier(random_state=42, n_estimators=100),
    "SVM": SVC(random_state=42, probability=True) # probability=True để có thể dùng cho các phương pháp đánh giá sau này
}

In [7]:
# Huấn luyện và đánh giá trên tập validation
trained_models = {}
for name, model in models.items():
    print(f"--- Đang huấn luyện mô hình: {name} ---")
    
    # Huấn luyện
    model.fit(X_train, y_train)
    
    # Dự đoán trên tập validation
    y_pred = model.predict(X_val)
    
    # Đánh giá
    accuracy = accuracy_score(y_val, y_pred)
    print(f"Accuracy trên tập validation: {accuracy:.4f}")
    print("Báo cáo phân loại (Classification Report):")
    print(classification_report(y_val, y_pred, target_names=target_names))
    
    # Lưu lại mô hình đã huấn luyện
    trained_models[name] = model
    print("-" * 40 + "\n")

--- Đang huấn luyện mô hình: Logistic Regression ---


ValueError: Found array with dim 4, while dim <= 2 is required by LogisticRegression.

In [ ]:
fig, axes = plt.subplots(1, len(models), figsize=(18, 5))

for i, (name, model) in enumerate(trained_models.items()):
    y_pred = model.predict(X_val)
    cm = confusion_matrix(y_val, y_pred)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i], 
                xticklabels=target_names, yticklabels=target_names)
    
    axes[i].set_title(f'Confusion Matrix - {name}')
    axes[i].set_xlabel('Predicted Label')
    axes[i].set_ylabel('True Label')

plt.tight_layout()
plt.savefig(f'{PATH_REPORTS_FIGURES}04_confusion_matrices.png')
plt.show()
